**Perturbation with 5,10 and 100% rate**

In [ ]:
import networkx as nx
import random

def perturbation_keep_clustering(sn_graph, prob_new_connection, prob_remove_connection, max_candidates=50):
    ins_connection = []
    del_connection = []

    for node in sn_graph.nodes:
        # Get neighbors and sample non-neighbors
        neighbors = set(sn_graph.neighbors(node))
        non_neighbors = [
            n for n in sn_graph.nodes if n not in neighbors and n != node
        ]
        non_neighbors_sample = random.sample(non_neighbors, min(len(non_neighbors), max_candidates))

        # Add edges to form new triangles
        if non_neighbors_sample and random.random() < prob_new_connection:
            for n in non_neighbors_sample:
                # Check if adding this edge forms a triangle
                shared_neighbors = neighbors & set(sn_graph.neighbors(n))
                if len(shared_neighbors) > 0:
                    sn_graph.add_edge(node, n)
                    ins_connection.append((node, n))
                    break  # Add only one edge per node in this iteration

        # Sample neighbors for edge removal
        neighbors_sample = random.sample(neighbors, min(len(neighbors), max_candidates))
        if neighbors_sample and random.random() < prob_remove_connection:
            for n in neighbors_sample:
                # Check if removing this edge preserves clustering
                shared_neighbors = neighbors & set(sn_graph.neighbors(n))
                if len(shared_neighbors) > 1:  # Removing this edge won't break triangles
                    sn_graph.remove_edge(node, n)
                    del_connection.append((node, n))
                    break  # Remove only one edge per node in this iteration

    return del_connection, ins_connection

def run_algo_with_different_perturbations(dataset_path):
    dataset = nx.read_gml(dataset_path)
    display_info(dataset)

    dataset_naive = nx.convert_node_labels_to_integers(dataset, first_label=1, ordering='default')
    original_clustering = nx.average_clustering(dataset_naive)

    perturbation_levels = [0.05, 0.1, 1.0]  # 5%, 10%, and 100% perturbation
    for prob in perturbation_levels:
        dataset_new = dataset_naive.copy()
        del_connection, new_connection = perturbation_keep_clustering(
            dataset_new, prob, prob
        )

        # Calculate and display clustering coefficients
        clustering_coeff_original = original_clustering
        clustering_coeff_new = nx.average_clustering(dataset_new)
        print(f"\n=== {int(prob * 100)}% Perturbation ===")
        print(f"Original Clustering Coefficient: {clustering_coeff_original}")
        print(f"New Clustering Coefficient: {clustering_coeff_new}")
        print("Graph changes:")
        print(f"  Edges added: {len(new_connection)}")
        print(f"  Edges removed: {len(del_connection)}")
        display_info(dataset_new)

def display_info(sn_graph):

    try:
        print(nx.info(sn_graph))
        betweenness = nx.betweenness_centrality(sn_graph, normalized=True, k=50)  # Reduce k for speed
        print("Betweenness centrality (average):", sum(betweenness.values()) / len(betweenness))
        closeness = nx.closeness_centrality(sn_graph)
        print("Closeness centrality (average):", sum(closeness.values()) / len(closeness))
        print("Number of isolates:", nx.number_of_isolates(sn_graph))
    except Exception as e:
        print("Error measuring graph properties:", e)

# Example usage
if __name__ == '__main__':
    # Provide the path to your GML dataset
    dataset_path = 'soc-Epinions1-converted.gml'  # Replace with different datasets
    run_algo_with_different_perturbations(dataset_path)


**Visualisation after and before 10 % of pertyrbation on Wiki-vote network**

In [ ]:
import random
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations

def visualize_triangles(graph, max_nodes=100, title="Graph Visualization Highlighting Triangles"):

    if graph.number_of_nodes() == 0:
        print("The graph is empty. Nothing to visualize.")
        return

    # Limit to a subgraph if the graph is too large
    if len(graph) > max_nodes:
        subgraph = graph.subgraph(list(graph.nodes)[:max_nodes])
        print(f"Graph is large, visualizing first {max_nodes} nodes.")
    else:
        subgraph = graph

    # Identify triangles in the graph
    triangle_edges = set()
    for node in subgraph.nodes:
        neighbors = set(subgraph.neighbors(node))
        for pair in combinations(neighbors, 2):
            if subgraph.has_edge(pair[0], pair[1]):  # Check if a triangle exists
                triangle_edges.update([(node, pair[0]), (node, pair[1]), (pair[0], pair[1])])

    pos = nx.spring_layout(subgraph)  # Spring layout
    plt.figure(figsize=(12, 12))

    nx.draw_networkx_nodes(subgraph, pos, node_color="lightblue", node_size=500, alpha=0.8)
    nx.draw_networkx_edges(subgraph, pos, edge_color="gray", alpha=0.5)
    nx.draw_networkx_labels(subgraph, pos, font_size=10)

    nx.draw_networkx_edges(
        subgraph,
        pos,
        edgelist=list(triangle_edges),
        edge_color="red",
        width=2,
        alpha=0.8
    )

    plt.title(title)
    plt.axis("off")
    plt.show()

def perturbation_10_percent_keep_clustering(sn_graph, prob_new_connection, prob_remove_connection, original_clustering, max_candidates=50):

    ins_connection = []
    del_connection = []

    for node in sn_graph.nodes:
        neighbors = set(sn_graph.neighbors(node))
        non_neighbors = [n for n in sn_graph.nodes if n not in neighbors and n != node]
        non_neighbors_sample = random.sample(non_neighbors, min(len(non_neighbors), max_candidates))

        # Add edges to form new triangles
        if non_neighbors_sample and random.random() < prob_new_connection:
            for n in non_neighbors_sample:
                shared_neighbors = neighbors & set(sn_graph.neighbors(n))
                if len(shared_neighbors) > 0:
                    sn_graph.add_edge(node, n)
                    ins_connection.append((node, n))
                    break

        # Sample neighbors for edge removal
        neighbors_sample = random.sample(neighbors, min(len(neighbors), max_candidates))
        if neighbors_sample and random.random() < prob_remove_connection:
            for n in neighbors_sample:
                shared_neighbors = neighbors & set(sn_graph.neighbors(n))
                if len(shared_neighbors) > 1:
                    sn_graph.remove_edge(node, n)
                    del_connection.append((node, n))
                    break

    return del_connection, ins_connection

def run_algo_with_visualization_and_clustering(dataset_path):
    dataset = nx.read_gml(dataset_path)
    dataset_naive = nx.convert_node_labels_to_integers(dataset, first_label=1, ordering='default')
    original_clustering = nx.average_clustering(dataset_naive)

    # Visualize before perturbation
    print("Visualizing BEFORE perturbation...")
    visualize_triangles(dataset_naive, max_nodes=100, title="Triangles Before Perturbation")
    print(f"Clustering Coefficient BEFORE perturbation: {original_clustering:.6f}")

    # Perform perturbation
    prob = 0.1  # 10% perturbation
    dataset_new = dataset_naive.copy()
    perturbation_10_percent_keep_clustering(
        dataset_new, prob, prob, original_clustering
    )

    # Visualize after perturbation
    print("Visualizing AFTER perturbation...")
    visualize_triangles(dataset_new, max_nodes=100, title="Triangles After Perturbation")
    new_clustering = nx.average_clustering(dataset_new)
    print(f"Clustering Coefficient AFTER perturbation: {new_clustering:.6f}")

if __name__ == '__main__':
    # Provide the path to your GML dataset
    dataset_path = 'wiki-Vote.gml'
    run_algo_with_visualization_and_clustering(dataset_path)


**Entropy of clustering and diameter of graph**

In [ ]:
import networkx as nx
import numpy as np
import random
from networkx.algorithms import approximation

def compute_entropy(attribute_values):
    """
    Compute the entropy of a given attribute distribution.
    Higher entropy means less variability (better anonymity).
    """
    unique, counts = np.unique(attribute_values, return_counts=True)
    probabilities = counts / len(attribute_values)
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy

def check_anonymity_and_diameter(graph, sample_threshold=1000):

    # Compute clustering coefficient

    clustering_coeffs = [nx.clustering(graph, n) for n in graph.nodes()]

    # Calculate entropy for anonymity
    degree_entropy = compute_entropy(degrees)
    clustering_entropy = compute_entropy(clustering_coeffs)

    # Calculate diameter
    if nx.is_connected(graph):
        if len(graph) > sample_threshold:
            # Approximate diameter for large graphs
            diameter = approximation.diameter(graph)
        else:
            diameter = nx.diameter(graph)
    else:
        largest_cc = max(nx.connected_components(graph), key=len)
        largest_cc_subgraph = graph.subgraph(largest_cc)

        if len(largest_cc_subgraph) > sample_threshold:
            # Approximate diameter for large connected component
            diameter = approximation.diameter(largest_cc_subgraph)
        else:
            diameter = nx.diameter(largest_cc_subgraph)

    print(f"Clustering Coefficient Entropy: {clustering_entropy:.4f}")
    print(f"Graph Diameter: {diameter}")

    return degree_entropy, clustering_entropy, diameter

def perturbation_keep_clustering(sn_graph, prob_new_connection, prob_remove_connection, max_candidates=50):

    ins_connection = []
    del_connection = []

    for node in sn_graph.nodes:
        neighbors = set(sn_graph.neighbors(node))
        non_neighbors = [n for n in sn_graph.nodes if n not in neighbors and n != node]
        non_neighbors_sample = random.sample(non_neighbors, min(len(non_neighbors), max_candidates))

        # Add edges to form new triangles
        if non_neighbors_sample and random.random() < prob_new_connection:
            for n in non_neighbors_sample:
                shared_neighbors = neighbors & set(sn_graph.neighbors(n))
                if len(shared_neighbors) > 0:
                    sn_graph.add_edge(node, n)
                    ins_connection.append((node, n))
                    break

        # Sample neighbors for edge removal
        neighbors_sample = random.sample(neighbors, min(len(neighbors), max_candidates))
        if neighbors_sample and random.random() < prob_remove_connection:
            for n in neighbors_sample:
                shared_neighbors = neighbors & set(sn_graph.neighbors(n))
                if len(shared_neighbors) > 1:
                    sn_graph.remove_edge(node, n)
                    del_connection.append((node, n))
                    break

    return del_connection, ins_connection

def run_analysis_with_perturbations(dataset_path):
    # Load the original graph
    original_graph = nx.read_gml(dataset_path)

    # Convert to undirected for diameter calculation
    undirected_graph = original_graph.to_undirected()

    dataset_naive = nx.convert_node_labels_to_integers(undirected_graph, first_label=1, ordering='default')

    # Original clustering coefficient
    original_clustering = nx.average_clustering(dataset_naive)

    # Check anonymity and diameter before perturbation
    print("=== Before Perturbation ===")
    print(f"Clustering Coefficient BEFORE: {original_clustering:.4f}")
    entropy_before, clustering_entropy_before, diameter_before = check_anonymity_and_diameter(dataset_naive)

    # perturbations
    perturbation_levels = [0.05, 0.1, 1.0]  # 5%, 10%, and 100% perturbation
    for prob in perturbation_levels:
        perturbed_graph = dataset_naive.copy()
        perturbation_keep_clustering(
            perturbed_graph, prob_new_connection=prob, prob_remove_connection=prob
        )

        # New clustering coefficient
        new_clustering = nx.average_clustering(perturbed_graph)

        # Check entropy and diameter after perturbation
        print(f"\n=== After {int(prob * 100)}% Perturbation ===")
        print(f"Clustering Coefficient AFTER: {new_clustering:.4f}")
       clustering_entropy_after, diameter_after = check_anonymity_and_diameter(perturbed_graph)

        print(f"\nSummary for {int(prob * 100)}% Perturbation:")
        print(f"Clustering Entropy Change: {clustering_entropy_before:.4f} → {clustering_entropy_after:.4f}")
        print(f"Diameter Change: {diameter_before} → {diameter_after}")

if __name__ == "__main__":
    dataset_path = "soc-Epinions1-converted.gml"  # Replace with different datasets
    run_analysis_with_perturbations(dataset_path)


**NMI SCORE**

In [ ]:
import networkx as nx
import random
import community as community_louvain
from sklearn.metrics import normalized_mutual_info_score
import matplotlib.pyplot as plt


def perturbation_keep_clustering(sn_graph, prob_new_connection, prob_remove_connection, max_candidates=50):

    for node in sn_graph.nodes:
        neighbors = set(sn_graph.neighbors(node))
        non_neighbors = [n for n in sn_graph.nodes if n not in neighbors and n != node]
        non_neighbors_sample = random.sample(non_neighbors, min(len(non_neighbors), max_candidates))

        # Add edges to form new triangles
        if non_neighbors_sample and random.random() < prob_new_connection:
            for n in non_neighbors_sample:
                shared_neighbors = neighbors & set(sn_graph.neighbors(n))
                if len(shared_neighbors) > 0:
                    sn_graph.add_edge(node, n)
                    break

        # Sample neighbors for edge removal
        neighbors_sample = random.sample(neighbors, min(len(neighbors), max_candidates))
        if neighbors_sample and random.random() < prob_remove_connection:
            for n in neighbors_sample:
                shared_neighbors = neighbors & set(sn_graph.neighbors(n))
                if len(shared_neighbors) > 1:
                    sn_graph.remove_edge(node, n)
                    break


def detect_communities(graph):

    return community_louvain.best_partition(graph)


def compare_communities(partition1, partition2):

    labels1 = list(partition1.values())
    labels2 = list(partition2.values())
    return normalized_mutual_info_score(labels1, labels2)


def visualize_communities(graph, partition, title="Community Structure"):

    pos = nx.spring_layout(graph)
    communities = set(partition.values())
    for community in communities:
        nodes = [node for node in partition if partition[node] == community]
        nx.draw_networkx_nodes(graph, pos, nodelist=nodes, node_size=50, label=f"Community {community}")
    nx.draw_networkx_edges(graph, pos, alpha=0.5)
    plt.title(title)
    plt.legend()
    plt.show()


def run_community_analysis(dataset_path):
    # Load the original graph
    original_graph = nx.read_gml(dataset_path)

    # Convert to undirected for community detection
    graph = original_graph.to_undirected()

    # Detect communities in the original graph
    original_partition = detect_communities(graph)
    print("=== Communities Detected in Original Graph ===")
    visualize_communities(graph, original_partition, title="Communities Before Perturbation")

    # Perform perturbation
    perturbation_levels = [0.05, 0.1, 1.0]  # 5%, 10%, and 100% perturbation
    for prob in perturbation_levels:
        perturbed_graph = graph.copy()
        perturbation_keep_clustering(perturbed_graph, prob_new_connection=prob, prob_remove_connection=prob)

        # Detect communities in the perturbed graph
        perturbed_partition = detect_communities(perturbed_graph)
        print(f"\n=== Communities Detected After {int(prob * 100)}% Perturbation ===")
        visualize_communities(perturbed_graph, perturbed_partition, title=f"Communities After {int(prob * 100)}% Perturbation")

        # Compare communities using NMI
        nmi_score = compare_communities(original_partition, perturbed_partition)
        print(f"NMI Score for {int(prob * 100)}% Perturbation: {nmi_score:.4f}")


if __name__ == "__main__":
    dataset_path = "wiki-Vote.gml"  # Replace with different datasets
    run_community_analysis(dataset_path)
